In [277]:
# for grabbing data

import numpy as np
import pandas as pd
import fukushima_telomere_methods as f_tm

import importlib
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---
##### &nbsp; 

The Fukushima Project seeks to determine whether regions contaminated by radiation from the Fukushima Daiichi Nuclear Disaster can be considered safe for human repopulation. Here, we use free-roaming >200 wild boar trapped from the contaminated regions as mammalian proxies for humans. On the boar we've collected >10 types of data from chemistry, physics (radiation), and molecular biology techniques to ascertain whether boar are negatively impacted by radiation in terms of various biometrics. We have data on snake as well.

The data was generated over >4 years and exists as tabular data in various formats. Our first aim is to read in all of the data, then clean/reformat it to enable combination of the various datasets. We've focused on standardizing the format for boar sample identifications (Sample ID) to enable combination of the data.

##### &nbsp; 
---

### Reading Aryn's snake data to dataframe

In [66]:
aryn_snake_df = pd.read_excel('./excel data/boar snake master template.xlsx', sheet_name=1)
aryn_snake_df.head(4)

,Sample ID,Concentration (ng/mcL),260/280,Sex,External Dose (uSv),Internal Dose (uSv),Telomere Length
0,CEC10,19.3,1.96,F,0.10,NaN,0.444298
1,CEC18,63.7,1.83,F,0.16,NaN,0.826190
2,CEC22,45.5,1.88,F,0.17,NaN,2.067214
3,CEC23,32.6,1.83,F,0.20,NaN,1.389182


In [142]:
# making column for radiation exposure status and encoded dummy variables from the new column

aryn_snake_df = f_tm.readable_snake_df_dummy_variables(aryn_snake_df)

### Saving Aryn's Snake dataframe for later retrieval

In [68]:
aryn_snake_df.to_csv('./excel data/aryn_snake_df.csv', index=False)

### Reading Aryn's Boar data to dataframe

In [73]:
aryn_boar_df = pd.read_excel('./excel data/boar snake master template.xlsx', sheet_name=0)
aryn_boar_df.head(4)

,Exposure Status,Sample ID,Sex,Age,Age Class,Estimated Age (months),External Dose (uSv),Internal Dose (uSv),qPCR ID,Concentration (ng/mcL),260/280,Normalized T/A Average (Average Telomere Length),Cortisol,Dicentrics (TC-FISH),PH anomalies
0,Control,180526C3,M,NaN,Yearling,15,0.38,NaN,C1,14.5,1.90,0.910559,NaN,NaN,NaN
1,Control,180528C2,F,NaN,Juv,8,-,NaN,C2,17.3,1.73,0.985085,NaN,NaN,NaN
2,Control,180604C2,F,NaN,Yearling,18,0.46,NaN,C3,15.8,1.90,1.117980,NaN,NaN,NaN
3,Control,180612C1,M,NaN,Yearling,15,0.27,NaN,C4,13.1,1.60,1.260040,NaN,NaN,NaN


### Cleaning Aryn's boar dataframe

In [145]:
# changing errant values to np.NaN
aryn_boar_df = aryn_boar_df.replace('-', np.NaN)

# encoding exposure status as dummy values
dummies = pd.get_dummies(aryn_boar_df['Exposure Status'], prefix='Encoded', drop_first=True)
aryn_boar_df['Encoded Exposed'] = dummies['Encoded_Exposed']

# cleaning Sample ID of dashes & spaces, + 'GPS' & 'collar' terms
aryn_boar_df['Sample ID'] = aryn_boar_df['Sample ID'].apply(lambda row: f_tm.remove_dashes_space_sampleIDs(row))

In [282]:
aryn_boar_df.to_csv('excel data/aryn_boar_df.csv', index=False)

### Reading the boar cortisol data

In [146]:
boar_cortisol_df = pd.read_excel('./excel data/boar cortisol.xlsx')
boar_cortisol_df.head(4)

,Biel #,Date Received,Sample Number (electronic sheet),Sample ID,Hair wt (mg),Cortisol (pg/mg),Re-run,MeOH Color,pH optimal for assay,Hair Type,Hair Length (cm),Color,Comments,"Worked on sample, but not able to process",Hair Took Extra Time,Repeats
0,1,43518,160804-1,160804-1,17.19,22.990111,N,N,normal,"course, short",3.81,black scalp and tip ends with tan middle,0,0,0,0
1,2,43518,161215 T-1,161215 T-1,25.28,29.120000,N,N,normal,course with split ends,5.02,black with tan tips and scalp end,0,0,0,0
2,3,43518,161215 T-2,161215 T-2,15.85,27.255521,N,N,normal,course with split ends,5.88,black with brown tips,0,0,0,0
3,4,43518,161130-1 O-210 collar,161130-1 O-210 collar,22.84,17.120000,N,N,normal,course with slightly split ends,7.02,black with brown tips,0,0,0,0


### Prep boar cortisol data for merging with other dataframes on Sample ID

In [206]:
# grabbing Sample IDs column in new dataframe, preserving index
prep_boar_cortisol_df = pd.DataFrame(boar_cortisol_df['Sample ID'])

# grabbing the Cortisol measurements & Sample Number (electronic sheet)
prep_boar_cortisol_df['Sample Number (electronic sheet)'] = boar_cortisol_df['Sample Number (electronic sheet)']
prep_boar_cortisol_df['Cortisol (pg/mg)'] = boar_cortisol_df['Cortisol (pg/mg)']

prep_boar_cortisol_df['Sample ID'] = prep_boar_cortisol_df['Sample ID'].astype('str')

# removing dashes/spaces & terms 'GPS' + 'collar' from sample IDs
prep_boar_cortisol_df['Sample ID'] = prep_boar_cortisol_df['Sample ID'].apply(lambda row: f_tm.remove_dashes_space_sampleIDs(row))

prep_boar_cortisol_df.head(4)

,Sample ID,Sample Number (electronic sheet),Cortisol (pg/mg)
0,1608041,160804-1,22.990111
1,161215T1,161215 T-1,29.120000
2,161215T2,161215 T-2,27.255521
3,1611301O210,161130-1 O-210 collar,17.120000


### Saving prepped cortisol data for later retrieval

In [175]:
prep_boar_cortisol_df.to_csv('excel data/prep_boar_cortisol_df.csv', index=False)

### Extracting Kelly's boar telomere FISH data

In [182]:
boar_teloFISH_list = f_tm.extract_boar_teloFISH_as_list('./excel data/')

Handling Telomeres - Fukushima Hyb 4 complete.xlsx...
Handling Telomeres - Fukushima Hyb 3 complete.xlsx...
Handling Telomeres - Fukushima Hyb 6 completed.xlsx...
Handling Meredith -- Telomeres - Fukushima Hyb 2.xlsx...
Handling Telomeres - Fukushima Hyb 1 complete.xlsx...
Handling Telomeres - Fukushima Hyb 5 complete.xlsx...
Finished collecting boar teloFISH data


### Turning data into properly formatted dataframe

In [280]:
# making a dataframe from the list
kelly_boar_teloFISH_df = pd.DataFrame(boar_teloFISH_list, columns=['Sample ID', 'teloFISH data', 'teloFISH means'])

# turning the teloFISH data, which is a series in each row containing 4800 individial telomere lengths, 
# into a list in each row so that it's saved & loaded properly
kelly_boar_teloFISH_df['teloFISH data'] = kelly_boar_teloFISH_df['teloFISH data'].apply(lambda row: row.tolist())
kelly_boar_teloFISH_df['Sample ID'] = kelly_boar_teloFISH_df['Sample ID'].apply(lambda row: f_tm.remove_dashes_space_sampleIDs(row))

kelly_boar_teloFISH_df.head(4)

,Sample ID,teloFISH data,teloFISH means
0,1606103,"[0.4747619790127278, 0.357467843021348, 0.1787...",0.439557
1,1607205,"[0.3183697976908881, 0.7037648159482789, 0.485...",0.452469
2,1607221,"[0.6590813355706104, 0.7484482963259474, 0.580...",0.533217
3,1607231,"[1.2343811454330924, 0.35188240797413944, 0.32...",0.583101


### Saving kelly boar teloFISH dataframe for later retrieval

In [281]:
kelly_boar_teloFISH_df.to_csv('./excel data/kelly_boar_teloFISH_df.csv', index=False)

### Loading Kelly boar dicentric data

In [202]:
kelly_boar_dicentrics = pd.read_excel('excel data/Fukushima Dicentrics.xlsx')

In [204]:
kelly_boar_dicentrics.rename(columns={'Pig ID': 'Sample ID'}, inplace=True)
kelly_boar_dicentrics['Sample ID'] = kelly_boar_dicentrics['Sample ID'].apply(lambda row: f_tm.remove_dashes_space_sampleIDs(row))
kelly_boar_dicentrics['Average Dicentrics per cell'] = kelly_boar_dicentrics['Dicentrics']/kelly_boar_dicentrics['Total Scored']                                       
kelly_boar_dicentrics.drop(['Dicentric Images', 'Notes', 'Dicentrics', 'Total Scored'], axis=1, inplace=True)

kelly_boar_dicentrics.head(4)

,Sample ID,Average Dicentrics per cell
0,1607261,0.01000
1,1608012,0.03000
2,1606101,0.04000
3,1611261O205,0.01087


### Saving kelly boar dicentrics dataframe for later retrieval

In [270]:
kelly_boar_dicentrics.to_csv('excel data/kelly_boar_dicentrics_df.csv', index=False)

### Loading New Radiation Exposure Calculations for Kelly Boar Data 

In [271]:
kelly_new_exposure_df = pd.read_excel("excel data/jared_editsKelly's boar_1Dec2018-edited1Aug19 (1).xlsx")

In [272]:
# print(kelly_new_exposure_df.columns)
kelly_new_exposure_df.head(4)

,ID,Place name,Unnamed: 2,Lat.,Lon.,"MEXT uSv/h at capture (Nov-16), midpoint of range at trap site","MEXT uSv/h @1m, from aircraft overfly","MEXT uSv/h @1m, max in 1000 m of capture site","MEXT uSv/h, @ 1m, max, at capture site","MEXT uSv/h, reasonable, midpoint of column I",...,"INTERNAL Life time dose, mGy 134","(from wolfram) Build-up, corrected, Integrated INTERNAL LIFE-TIME dose mGy Cs134","INTERNAL Life time dose, mGy 137","INTERNAL Life time dose, mGy 134+137","EXTERNAL Dose Reasonable Life Time, mGy (134+137)",% of INTERNAL to EXTERNAL dose,"EXTERNAL Dose Maximum Life Time, mGy (134+137)","Reasonable Total Life Time Dose mGy (Int+Ext,134+137)","Maximum Total Life Time Dose mGy (Int+Ext,134+137)",Dose rate (uGy/h) at time of capture (Int+Ext; 134+137)
0,160610-1,Namie,Akougi-Koakuto,37.568932,140.778014,7.0,3.8-9.5,9.5,9.5,9.5,...,0.599879,0.672739,1.856921,2.529660,57.456,4.40278,57.456,59.985660,59.985660,7.406217
1,160610-2,Namie,Sakai-Matsukiuchi,37.479072,140.979392,3.0,3.8-9.5,9.5-19,19.0,14.0,...,0.747327,0.812612,2.482790,3.295402,61.152,5.38887,82.992,64.447402,86.287402,3.739496
2,160610-3,Namie,Sakai-Matsukiuchi,37.479072,140.979392,3.0,3.8-9.5,9.5-19,19.0,14.0,...,0.736103,0.808553,2.410267,3.218820,68.208,4.71912,92.568,71.426820,95.786820,3.645807
3,160720-5,Fukushima,Yamaguchi-Gohonmatsu,37.761529,140.501009,0.7,0.5-1.0,0.5-1.0,1.0,0.7,...,0.0119858,0.012258,0.036077,0.048335,0.882,5.48015,1.260,0.930335,1.308335,0.738145


In [278]:
cols_to_merge = ['ID', 'Sex', 'Age (weeks)',
'INTERNAL uGy/h Cs-134', 'INTERNAL uGy/h Cs-137',
'INTERNAL Life time dose,   mGy 134',
'(from wolfram) Build-up, corrected, Integrated INTERNAL LIFE-TIME dose mGy  Cs134',
'INTERNAL Life time dose, mGy 137',
'INTERNAL Life time dose, mGy 134+137',
'EXTERNAL Dose Reasonable Life Time, mGy (134+137)',
'% of INTERNAL to EXTERNAL dose',
'EXTERNAL Dose Maximum Life Time, mGy (134+137)',
'Reasonable Total Life Time Dose mGy (Int+Ext,134+137)',
'Maximum Total Life Time Dose mGy (Int+Ext,134+137)',
'Dose rate (uGy/h) at time of capture (Int+Ext; 134+137)']

trim_kelly_new_exposure_df = pd.DataFrame(kelly_new_exposure_df[cols_to_merge])
(trim_kelly_new_exposure_df.rename(columns={'ID':'Sample ID', 
'(from wolfram) Build-up, corrected, Integrated INTERNAL LIFE-TIME dose mGy  Cs134':'Integrated INTERNAL LIFE-TIME dose mGy  Cs134'}, inplace=True))
trim_kelly_new_exposure_df['Sample ID'] = trim_kelly_new_exposure_df['Sample ID'].apply(lambda row: f_tm.remove_dashes_space_sampleIDs(row))

trim_kelly_new_exposure_df.head(4)

,Sample ID,Sex,Age (weeks),INTERNAL uGy/h Cs-134,INTERNAL uGy/h Cs-137,"INTERNAL Life time dose, mGy 134",Integrated INTERNAL LIFE-TIME dose mGy Cs134,"INTERNAL Life time dose, mGy 137","INTERNAL Life time dose, mGy 134+137","EXTERNAL Dose Reasonable Life Time, mGy (134+137)",% of INTERNAL to EXTERNAL dose,"EXTERNAL Dose Maximum Life Time, mGy (134+137)","Reasonable Total Life Time Dose mGy (Int+Ext,134+137)","Maximum Total Life Time Dose mGy (Int+Ext,134+137)",Dose rate (uGy/h) at time of capture (Int+Ext; 134+137)
0,1606101,Male,33-39,0.0991863,0.307031,0.599879,0.672739,1.856921,2.529660,57.456,4.40278,57.456,59.985660,59.985660,7.406217
1,1606102,Female,26,0.171091,0.568404,0.747327,0.812612,2.482790,3.295402,61.152,5.38887,82.992,64.447402,86.287402,3.739496
2,1606103,Male,27-31,0.151089,0.494718,0.736103,0.808553,2.410267,3.218820,68.208,4.71912,92.568,71.426820,95.786820,3.645807
3,1607205,Male,6-9,0.00951251,0.028632,0.0119858,0.012258,0.036077,0.048335,0.882,5.48015,1.260,0.930335,1.308335,0.738145


### Saving Kelly boar new exposure data for later retrieval

In [279]:
trim_kelly_new_exposure_df.to_csv('excel data/trim_kelly_new_exposure_df.csv', index=False)

In [236]:
df_list = [trim_kelly_new_exposure_df, prep_boar_cortisol_df]

for df in df_list:
    f_tm.count_shared_sample_IDs(kelly_boar_teloFISH_df, df, print_names='no')

The number of sample sample IDs are: 38
The number of sample sample IDs are: 14


In [238]:
kelly_teloFISH_IDs = list(kelly_boar_teloFISH_df['Sample ID'].unique())
kelly_exposure_IDs = list(trim_kelly_new_exposure_df['Sample ID'].unique())

a = kelly_teloFISH_IDs
b = kelly_exposure_IDs

mismatched = [sample_ID for sample_ID in a if sample_ID not in b]
print(mismatched)

['161209O239', '161209O240', '161209O242']
